#### Importing libs

In [1]:
import pandas as pd
import numpy as np

# Data Import

##### Importing features

In [2]:
sheet_id = "1xOPRcRkJrGzxb65dieEHrWHnqXYrrSpx7Ack3pr6Pdw"
features = pd.read_excel(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx", sheet_name = None)
print(features.keys()) # Empresas

dict_keys(['BOMBRIL', 'COSAN', 'METALFRIO', 'TAESA', 'TUPY', 'JHSF', 'KLABIN', 'VIBRA', 'VALE', 'RANDON', 'IRANI', 'CYRELA', 'UNIPAR', 'SPRINGS', 'HAGA', 'DIRECIONAL', 'SIDERURGICA ALIPERTI', 'AZEVEDO E TRAVASSOS', 'PANATLANTICA', 'TELEFONICA', 'ALUPAR', 'PETRORIO', 'DOMMO', 'MRV', 'HOTEIS OTHON', 'JOÃO FORTES', 'MANGELS', 'ENAUTA', 'FRAS-LE', 'NATURA', 'PETTENATTI', 'IMC', 'NUTRIPLANT', 'OCEANPACT', 'AMERICANAS', 'PBG', 'ROSSI', 'TEKNO', 'TIM', 'ENERGISA', '3R PETROLEUM', 'ENEVA', 'ETERNIT', 'EUCATEX', 'FER HERINGER', 'GPC PART', 'LUPATECH', 'MMX MINER', 'OI', 'OSX BRASIL', 'PDG REALT', 'PET MANGUINHOS', 'POMIFRUTAS', 'RECRUSUL', 'REDE ENERGIA', 'SANSUY', 'SARAIVA', 'TECNOSOLO', 'TEKA', 'VIVER', 'WETZEL', 'BRASKEM', 'CEMIG', 'DURATEX', 'EZTEC', 'INDS ROMI', 'LITEL', 'MARCOPOLO', 'METAL IGUAÇU', 'SLC AGRICOLA', 'SONDOTECNICA', 'ULTRAPAR'])


##### Importing targets

In [3]:
sheet_id = "116cM2eSTve3UHHYESOdWXRiYxx8gGAPPLlonNDNIn6M"
target = pd.read_excel(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")

# Alterando índices
target.index = target["Empresa"]
target.drop(["Empresa"], axis = 1, inplace = True)

# Montando o alvo
print("Classificações: ",list(set(target["Classificação"])))
target["Recuperação Judicial"] = np.where(target["Classificação"] == "Recuperação Judicial", 1, 0)
target["Amostra Falida"] = np.where(target["Classificação"] == "Amostra Falida", 1, 0)
target["Alvo"] = target["Amostra Falida"] + target["Recuperação Judicial"]
target.drop(["Recuperação Judicial", "Amostra Falida"], axis = 1, inplace = True)
target.head()

Classificações:  ['Não Falida', 'Recuperação Judicial', 'Amostra Par', 'Amostra Falida']


,Ticker,Setor,Classificação,Data do Evento,Alvo
Empresa,,,,,
3R PETROLEUM,RRRP,"Petróleo, Gás e Biocombustíveis",Não Falida,2013-01-17,0
ALUPAR,ALUP,Utilidade Pública,Amostra Par,2014-12-09,0
AMERICANAS,AMER,Consumo Cíclico,Não Falida,2018-11-23,0
AVIANCA (OCEANAIR),NaN,Bens Industriais,Amostra Falida,NaT,1
AZEVEDO E TRAVASSOS,AZEV,Bens Industriais,Amostra Par,2012-08-03,0


##### Validation

In [4]:
res = 0
miss = []
for i in list(features.keys()):
    if i in list(target.index):
        res += 1
    else:
        miss.append(i)

print("Número de Empresas (Target):", res)
print("Número de Empresas (Features):", len(features.keys())) # Numero de empresas
print("Empresas Faltando:", miss)

Número de Empresas (Target): 72
Número de Empresas (Features): 72
Empresas Faltando: []


# Data Wrangling

Próximos passos: 
* tratar NA's

In [5]:
# Corte Transversal ou Cross-Sectional (analisa dados em um momento específico) -> oposto do longitudinal (série temporal)
indicadores = list(features["BOMBRIL"].iloc[35:, 0])

def wrangle(ano_pre_fal):
    # monta df para 1 a 5 anos pré-falência
    
    df = pd.DataFrame(index = features.keys())

    for indicador in indicadores:
        coluna = []
        for empresa in features.keys():
            data = features[empresa]
            inds = data.columns[0]
            val_ind = data[data[inds] == indicador].iloc[0, 6-ano_pre_fal]
            coluna.append(val_ind)

        df[indicador] = coluna
    
    df.replace('#DIV/0!', np.nan, inplace = True)
    df.replace(0, np.nan, inplace = True)
    df = df.join(target["Alvo"]) # Acrescenta os alvos

    return df

In [6]:
df_1y = wrangle(1)
df_1y.head()

,Dispon/PC,Dispon/Receita Liq Operacional,Dispon/AC,Dispon/AT,AC/PC,AC/AT,PC/AT,PC/Receita Liq Operacional,Estoque/AT,Quick assets/AT,...,VA/IMOBILIZADO,VA/AT,VA/RLO,Contas a pagar/RLO,AC/RLO,Estoque/RLO,NOWC/RLO,Contas a receber/RLO,RLO/AT,Alvo
BOMBRIL,0.037400,0.022244,0.101842,0.029547,0.367232,0.290121,0.790021,0.594763,0.031404,0.258718,...,1.912136,0.573094,0.431451,0.095822,0.218416,0.023642,-0.112902,0.130422,1.328294,1
COSAN,1.644152,0.042668,0.535983,0.063081,3.067545,0.117692,0.038367,0.025951,0.012073,0.105619,...,-94.153913,-0.273755,-0.185168,0.004062,0.079607,0.008166,0.0563916,0.009919,1.478415,0
METALFRIO,0.714739,0.581478,0.562656,0.390647,1.270295,0.694291,0.546559,0.813553,0.154425,0.539866,...,-0.557492,-0.091249,-0.135825,0.192842,1.033452,0.229861,0.795228,0.176731,0.671817,0
TAESA,0.621887,0.963256,0.842059,0.172986,0.738532,0.205432,0.278163,1.548924,NaN,0.205432,...,64.438054,0.164443,0.915685,0.014210,1.143930,NaN,1.12769,0.178644,0.179584,0
TUPY,1.260625,0.444879,0.556159,0.265093,2.266661,0.476648,0.210287,0.352903,0.067499,0.409149,...,-0.199108,-0.066526,-0.111644,0.086110,0.799912,0.113277,0.613744,0.158185,0.595876,0


# Tratamento de NA's

In [7]:
print("DF Original:", df_1y.shape[0], "empresas")
print("DF sem NA's:", df_1y.dropna().shape[0], "empresas")

DF Original: 72 empresas
DF sem NA's: 50 empresas


In [8]:
# Indicadores com maior quantidade de valores faltantes
df_1y.isna().sum().sort_values(ascending = False)

Juros/VA                               13
Divida LP/AT                           12
Divida LP/PL                           12
Estoque/RLO                            11
Estoque/AT                             11
Juros/Ativo Total                      10
Divida total/AT                        10
Divida total/PL                        10
Juros/Lucro Liquido                    10
Juros/EBITDA                           10
Dívida financeira/FCFF                 10
VA/IMOBILIZADO                          7
Contas a pagar/RLO                      7
Contas a receber/RLO                    7
PC/Receita Liq Operacional              6
RLO/AT                                  6
Dispon/Receita Liq Operacional          6
Capital de giro/Rec Liq Operacional     6
FCFF/Rec Liq Operacional                6
EBITDA/Rec Liq Operacional              6
NOWC/RLO                                6
AC/RLO                                  6
VA/RLO                                  6
Lucro liquido/RLO                 

In [9]:
print("Classificação das empresas que contém pelo menos um NA:")
print(df_1y[df_1y.isna().any(axis=1)]["Alvo"].value_counts())
df_1y[df_1y.isna().any(axis=1)]

Classificação das empresas que contém pelo menos um NA:
0    16
1     6
Name: Alvo, dtype: int64


,Dispon/PC,Dispon/Receita Liq Operacional,Dispon/AC,Dispon/AT,AC/PC,AC/AT,PC/AT,PC/Receita Liq Operacional,Estoque/AT,Quick assets/AT,...,VA/IMOBILIZADO,VA/AT,VA/RLO,Contas a pagar/RLO,AC/RLO,Estoque/RLO,NOWC/RLO,Contas a receber/RLO,RLO/AT,Alvo
METALFRIO,0.714739,0.581478,0.562656,0.390647,1.270295,0.694291,0.546559,0.813553,1.544245e-01,0.539866,...,-0.557492,-0.091249,-0.135825,0.192842,1.033452,0.229861,0.795228,0.176731,0.671817,0
TAESA,0.621887,0.963256,0.842059,0.172986,0.738532,0.205432,0.278163,1.548924,NaN,0.205432,...,64.438054,0.164443,0.915685,0.014210,1.143930,NaN,1.12769,0.178644,0.179584,0
RANDON,0.381375,0.088838,0.218274,0.146633,1.747228,0.671782,0.384484,0.232943,1.901108e-01,0.481671,...,0.226641,0.052748,0.031958,0.050101,0.407004,0.115180,0.29389,0.150819,1.650554,0
SIDERURGICA ALIPERTI,0.608696,0.536082,0.250689,0.043128,2.428094,0.172038,0.070853,0.880707,8.530806e-02,0.086730,...,0.021410,0.016128,0.200476,0.070692,2.138439,1.060383,1.64359,0.129602,0.080450,0
AZEVEDO E TRAVASSOS,0.193182,0.045285,0.098039,0.061999,1.970455,0.632385,0.320934,0.234417,9.117433e-02,0.541211,...,-0.534925,-0.156069,-0.113996,0.047416,0.461907,0.066596,0.345232,0.323921,1.369074,0
PANATLANTICA,0.982173,0.323818,0.445514,0.334974,2.204584,0.751882,0.341054,0.329695,1.783439e-01,0.573538,...,-0.308102,-0.093394,-0.090283,0.101595,0.726840,0.172404,#VALUE!,0.201511,1.034453,0
PETRORIO,1.910714,97.272727,0.635958,0.237119,3.004464,0.372853,0.124100,50.909091,NaN,0.372853,...,-0.171415,-0.013200,-5.415170,14.318182,152.954545,NaN,77.2727,0.227273,0.002438,0
DOMMO,501.020619,NaN,0.971032,0.031559,515.967010,0.032500,0.000063,NaN,NaN,0.032500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
ENAUTA,10.604677,2.059918,0.865649,0.391249,12.250557,0.451972,0.036894,0.194246,3.861956e-03,0.448110,...,0.189844,0.060307,0.317515,0.070301,2.379624,0.020333,2.28748,0.278823,0.189934,0
NATURA,0.607874,0.169150,0.417297,0.288275,1.456693,0.690814,0.474235,0.278265,1.135176e-01,0.577296,...,3.873360,1.009561,0.592376,0.067046,0.405346,0.066608,0.307187,0.138475,1.704257,0


# Exportando a Base

In [10]:
sheet_names = ["df_" + str(i) + "y" for i in range(1,6)]
dfs = [wrangle(i) for i in range(1,6)]

writer = pd.ExcelWriter('../data/dados.xlsx', engine='xlsxwriter')
for n in range(1,6): dfs[n-1].to_excel(writer, sheet_name = sheet_names[n-1], index_label = "Empresa")
writer.save()

FileCreateError: [Errno 13] Permission denied: '../data/dados.xlsx'